In [2]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import selenium
from splinter import Browser
from selenium import webdriver

In [3]:
# collect links to tables of UFO sightings by year

# create a path to chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

# create a variable for the url
url = 'https://nuforc.org/webreports/ndxpost.html'

# visit the url
browser.visit(url)

# create a variable for the html
html = browser.html

# create a beautiful soup object
soup = bs(html, 'html.parser')

# find all the links on the page
links = soup.find_all('a',)

# close the browser
browser.quit()


In [4]:
urls =[ x['href'] for x in links][1:]

In [5]:
UFO_df = pd.DataFrame()
completed_pages = 0
failed_pages = 0

# Iterate through each link
for link in urls:
    # Get the URL of the linked page
    url = "https://nuforc.org/webreports/" + link
    try:
        # Make a request to the linked page
        response = requests.get(url)

        if not response.ok:
            print("Nothing on page to load")
            failed_pages += 1
            break
        # Extract the data from the table and store it in a dataframe
        table_df = pd.read_html(response.text)[0]
        # Append the dataframe to the list of dataframes
        UFO_df = pd.concat([UFO_df, table_df], axis=0)
        completed_pages += 1
    except:
        print('error')
        break

print(f"Completed {completed_pages} pages, failed {failed_pages} pages")


Completed 619 pages, failed 0 pages


In [8]:
# fill in missing values with 'Unknown'
UFO_df.fillna('Unknown', inplace=True)

In [41]:
UFO_df.to_json('UFO_data.csv', index=False)

#### Additional ETL

In [9]:
# split dataframe into cleaning dataframes

# define the regular expression pattern to match the last word
pattern = r'\b(\w+)\b$'

#find the longest encounters first
hours = UFO_df.loc[UFO_df['Duration'].str.contains('hour')].copy()
# get the minutes dataframe by removing the hours
minutes = UFO_df.loc[UFO_df['Duration'].str.contains('minute') & ~UFO_df['Duration'].str.contains('hour')].copy()
# get the seconds dataframe by removing the minutes
seconds = UFO_df.loc[UFO_df['Duration'].str.contains('second') & ~UFO_df['Duration'].str.contains('minute') & ~UFO_df['Duration'].str.contains('hour')].copy()
# get the unknown dataframe by removing the seconds
unknown = UFO_df.loc[~UFO_df['Duration'].str.contains('second') & ~UFO_df['Duration'].str.contains('minute') & ~UFO_df['Duration'].str.contains('hour')].copy()



In [31]:
hours['unit'] = hours['Duration'].str.extract(pattern, expand=True, )
hours.unit.fillna('NaN', inplace=True)

/var/folders/9_/j1y8d7d16j72pf4nkz7l4y9w0000gn/T/ipykernel_65880/2518253172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hours['unit'] = hours['Duration'].str.extract(pattern, expand=True, )
/var/folders/9_/j1y8d7d16j72pf4nkz7l4y9w0000gn/T/ipykernel_65880/2518253172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hours.unit.fillna('NaN', inplace=True)


In [35]:
hours.loc[~(hours.unit.str.contains('hour')) & (hours.unit != 'NaN') ]

,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images,unit
147,3/14/23 03:00,Theodore,AL,USA,Star,2 hours. 0300-daylight,"Two clusters of UAP?, The first cluster starte...",4/9/23,Yes,daylight
293,7/15/20 02:00,Fllorence,AL,USA,Chevron,For a I half hour or so,Was leaving friends home. Saw bright lite in s...,4/9/23,Yes,so
39,3/1/23 20:00,Kalamazoo,MI,USA,Unknown,Over an hour and still th,2 lights. NUFORC Note: Jupiter and Venus,3/6/23,Unknown,th
52,3/1/23 19:35,Laguna Woods,CA,USA,Circle,Half hour so far,"Saw two stationary lights, one brighter than t...",3/6/23,Yes,far
254,2/12/23 21:00,Minot,ND,USA,Circle,Few hours then it left,It looked like a big star. But it was moving.,3/6/23,Yes,left
...,...,...,...,...,...,...,...,...,...,...
697,4/18/96 17:40,"Brisbane (near) (Queensland, Australia)",Unknown,Australia,Unknown,5 hours plus,"Five hours plus interaction with an ""invisible...",1/28/99,Unknown,plus
85,6/26/98 10:15,Trumann/Jonesboro,AR,USA,Other,1 hour 45 mins,"There was a full moon this night, so the shape...",11/21/98,Unknown,mins
45,11/6/98 07:00,Bothell,WA,USA,Oval,half hour plus,Saw 2 cylindrical and 4 oval objects slowly de...,11/19/98,Unknown,plus
212,1975 aprx 20:00,San Gabriel,CA,USA,Cigar,hour or so,"Cigar or oblong shaped object, just sat there ...",11/1/98,Unknown,so
